In [1]:
#Import necessary  libraries
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
import librosa.display
import os
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os

#glob used to return all file paths that match a specific pattern
from glob import glob


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install tensorflow

     |████▎                           | 65.0 MB 2.2 MB/s eta 0:03:14

In [ ]:
dbs = [-6, 0, 6]
kinds = ['normal', 'abnormal']
file_name = ['00','02','04','06']
def name_path(name):r
    paths = []

    for db in dbs:
        for i in range(4):
            paths.append(f'/Users/yusufakcakaya/Downloads/{db}_dB_valve/id_0{i*2}/{name}')
    return paths
    


In [ ]:
file_names = {'normal': name_path(kinds[0]),'abnormal': name_path(kinds[1])}



In [ ]:
def to_png(files,name_dir,name_file):
        
    files = glob(files + '/*.wav')
    
    
    for file in range(101):
        data, sr = librosa.load(files[file])
        data = scale(data)
       
        countstr = str(file)

        melspec = librosa.feature.melspectrogram(y=data, sr=sr, n_mels=128)
           
        log_melspec = librosa.power_to_db(melspec, ref=np.max)  
        librosa.display.specshow(log_melspec, sr=sr)
            
        if file < 80:

            # save to png
            directory = name_dir
            if not os.path.exists(directory):
                os.makedirs(directory)
        
            png_number= name_file + countstr  


            plt.savefig(directory + '/' + (png_number) + '.png')
        elif file > 80:
            
             # save to png
            directory = name_dir + '_validation'
            if not os.path.exists(directory):
                os.makedirs(directory)
        
            png_number= name_file + countstr  


            plt.savefig(directory + '/' + (png_number) + '.png')
            
    
    return 0
      

In [ ]:
db = ['-6','0','6']
kinds = ['normal', 'abnormal']
file_name = ['00','02','04','06']

In [ ]:
#index = 0
#for b in kinds:
#    for i in db:
#        for a in file_name:
#            png_name = i+a
#            
#            to_png(file_names[b][index],b,png_name)
            
#            index += 1
#            if index == 12:
#                index = 0

Define the path to our data. Let’s define a function called get_data() that makes it easier for us to create our train and validation dataset. We define the two labels ‘normal’ and ‘abnormal’ that we will use. We use the Opencv imread function to read the images in the RGB format and resize the images to our desired width and height in this case both being 224.

In [ ]:
labels = ["normal", "abnorm"]
img_size = 224

def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)   # create path 
        class_num = labels.index(label)        # get the classification  (0 or a 1). 0=Normal 1=Abnormal
        for img in os.listdir(path):           # iterate over each image per two of them
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
#fetch our train and validation data
from sklearn.model_selection import train_test_split
data = get_data("valve")
x_train, x_test, y_train, y_test = train_test_split(data[:,0], data[:,1], test_size = 0.2, random_state=42)

In [ ]:
print(train[0,0].shape)
print(test[0,0].shape)

In [ ]:
print('Train len',len(train))
print('Test len',len(test))

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in test:
  x_test.append(feature)
  y_test.append(label)

# Normalize the data
from sklearn.preprocessing import Normalizer
norm = Normalizer()
x_train = norm.fit_transform(x_train)
x_test = norm.transform(x_test)
#x_train = np.array(x_train) / 255
#x_test = np.array(x_test) / 255

#x_train.reshape(-1, img_size, img_size, 1)
#y_train = np.array(y_train)

#x_test.reshape(-1, img_size, img_size, 1)
#y_test = np.array(y_test)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['recall'])

history = model.fit(x_train,y_train,epochs = 500 , validation_data = (x_test, y_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(500)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()